<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# A survey of some of the `NRPy+` features

## Author: Thiago Assumpção


## Introduction to some of the recurring NRPy+ features with practical examples.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#initializenrpy): Initialize core Python/NRPy+ modules
1. [Step 2](#coord): Coordinate systems
1. [Step 3](#indexed_expressions): Indexed expressions
1. [Step 4](#fd_derivatives): Finite-difference derivatives
1. [Step 5](#c_code_generation): Generating C code
1. [Step 6](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='initializenrpy'></a>

# Step 1: Initialize core Python/NRPy+ modules \[Back to [top](#toc)\]
$$\label{initializenrpy}$$

In [1]:
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import reference_metric as rfm   # NRPy+: Reference metric support
import finite_difference as fin  # NRPy+: Finite difference C code generation module
import grid as gri               # NRPy+: Functions having to do with numerical grids
from outputC import lhrh         # NRPy+: Core C code output module
from UnitTesting.assert_equal import check_zero # NRPy+: unit testing suite

<a id='coord'></a>

# Step 2: Coordinate systems \[Back to [top](#toc)\]
$$\label{coord}$$

There are several curvilinear, orthogonal coordinate systems implemented in `reference_metric` module. These are the ones currently available:

```
Spherical-like coordinate systems:
Spherical
SinhSpherical
SinhSphericalv2

Cylindrical-like coordinate systems:
Cylindrical
SinhCylindrical
SinhCylindricalv2

Cartesian-like coordinate systems:
Cartesian
SinhCartesian

Prolate spheroidal coordinates:
SymTP
SinhSymTP
```


Across the NRPy+ code base, it is customary to import this module as
```
import reference_metric as rfm
```

`NRPy+` assumes all curvilinear coordinate systems map directly from a uniform numerical grid with coordinates (`xx[0]`,`xx[1]`,`xx[2]`). Thus when defining reference metrics, all defined coordinate quantities must be in terms of the `xx[]` array. As we will see, this adds a great deal of flexibility.

For instance, if the chosen coordinate system in Cartesian:
* $x = xx_0$
* $y = xx_1$
* $z = xx_2$


If the chosen coordinate system in spherical: 
* $r = xx_0$
* $\theta = xx_1$
* $\phi = xx_2$

In [2]:
# Let us set choose the coordinate system
par.set_parval_from_str("reference_metric::CoordSystem", "Spherical")
# And set the dimension to 3
par.set_parval_from_str("grid::DIM",3)
# Next, we compute several geometrical quantities, such as Christoffel symbols, scale factors, metric tensor, etc...
rfm.reference_metric()

We can access the metric tensor via `rfm.ghatDD`. The hat diacritic denotes a flat metric.

In [3]:
# Metric tensor
sp.Matrix(rfm.ghatDD)

Matrix([
[1,      0,                  0],
[0, xx0**2,                  0],
[0,      0, xx0**2*sin(xx1)**2]])

In [4]:
# Christoffel symbols
sp.Matrix(rfm.GammahatUDD[0])

Matrix([
[0,    0,                0],
[0, -xx0,                0],
[0,    0, -xx0*sin(xx1)**2]])

In [5]:
# Christoffel symbols
sp.Matrix(rfm.GammahatUDD[1])

Matrix([
[    0, 1/xx0,             0],
[1/xx0,     0,             0],
[    0,     0, -sin(2*xx1)/2]])

In [6]:
# Christoffel symbols
sp.Matrix(rfm.GammahatUDD[2])

Matrix([
[    0,                          0,                      1/xx0],
[    0,                          0, sin(2*xx1)/(2*sin(xx1)**2)],
[1/xx0, sin(2*xx1)/(2*sin(xx1)**2),                          0]])

In [7]:
# Converting to Cartesian coordinates
rfm.xx_to_Cart

[xx0*sin(xx1)*cos(xx2), xx0*sin(xx1)*sin(xx2), xx0*cos(xx1), 0]

In [8]:
# Converting from Cartesian coordinates
rfm.Cart_to_xx

[sqrt(Cartx**2 + Carty**2 + Cartz**2),
 acos(Cartz/sqrt(Cartx**2 + Carty**2 + Cartz**2)),
 atan2(Carty, Cartx),
 0]

You can set a different coordinate system and run the above cells again to see the results.

<a id='indexed_expressions'></a>

# Step 3: Indexed expressions \[Back to [top](#toc)\]
$$\label{indexed_expressions}$$

To manipulate expressions with indices, such as tensors or pseudotensors, we use the `indexedexp` module. Across the NRPy+ code base, it is customary to import this module as
```
import indexedexp as ixp
```

**Naming convention**: 

For each upper index, we append the character `U` at the end of the variable name. Likewise, for each lower index, we append the character `D` at the end of the variable name.

**Examples**:

* $a^{i} \leftrightarrow$ `aU`


* $b_{i} \leftrightarrow$ `bD`


* $g_{ij} \leftrightarrow$ `gDD`


* $\Gamma^{i}_{ \ jk} \leftrightarrow$ `GammaUDD`


* $R^{i}_{ \ j k l m} \leftrightarrow$ `RUDDDD`

Do not use `D` or `U` as variable names, as to avoid unexpected behavior.

To define an indexed expression, we need to provide the dimension, `DIM`, and the symmetry option, `sym` (if the rank is higher than 1).

In [9]:
import indexedexp as ixp

Declaring a vector in three dimentions:

In [10]:
aU = ixp.declarerank1(symbol='aU', DIM=3)

In [11]:
for i in range(3):
    print(f"aU[{i}] = {aU[i]}")

aU[0] = aU0
aU[1] = aU1
aU[2] = aU2


It is also customary, but not mandatory, to use the same characters to denote the symbol and the variable name. Note that the symbol of each component of the tensor has the index appended at the end.

**Common pitfall**: you cannot use the variable `aU0` directly, as it can only be accessed via `aU[0]`. If you wish to use `aU0`, you must define it as a SymPy variable. Also notice that `aU[0]` is just a normal SymPy symbol:

In [12]:
type(aU[0])

sympy.core.symbol.Symbol

In [13]:
aU0 = sp.symbols('aU0')

In [14]:
print(aU[0] - aU0)

0


In [15]:
print("They have the same id:")
print( hex(id(aU0)) )
print( hex(id(aU[0])) )

They have the same id:
0x7efde2ad4f00
0x7efde2ad4f00


Now let us define a vector with each component equal to 0. There is no need to define a symbol:

In [16]:
bD = ixp.zerorank1(DIM=3)

In [17]:
for i in range(3):
    print(f"bD[{i}] = {bD[i]}")

bD[0] = 0
bD[1] = 0
bD[2] = 0


Defining a rank-2 tensor:

In [18]:
gDD = ixp.declarerank2('gDD', 'sym01', DIM=3)

In [19]:
gDD

[[gDD00, gDD01, gDD02], [gDD01, gDD11, gDD12], [gDD02, gDD12, gDD22]]

In [20]:
# Checking symmetry
for i in range(3):
    for j in range(i+1,3):
         print(f"gDD[{i}][{j}] - gDD[{j}][{i}] = {gDD[i][j] - gDD[j][i]}")

gDD[0][1] - gDD[1][0] = 0
gDD[0][2] - gDD[2][0] = 0
gDD[1][2] - gDD[2][1] = 0


Index-lowering example:

$$
a_i = g_{ij}a^{j} \,.
$$

In [21]:
# First, define aD
aD = ixp.zerorank1(DIM=3) # we can define with 0's, since these components will be updated
# Index lowering
for i in range(3):
    for j in range(3):
        aD[i] += gDD[i][j]*aU[j]

In [22]:
aD

[aU0*gDD00 + aU1*gDD01 + aU2*gDD02,
 aU0*gDD01 + aU1*gDD11 + aU2*gDD12,
 aU0*gDD02 + aU1*gDD12 + aU2*gDD22]

Defining a tensor $T_{ijkl}$ with the following symmetries:

$$
\begin{aligned}
T_{ijkl} &= - T_{ljki} \,, \\
T_{ijkl} &= T_{ikjl} \,. \\
\end{aligned}
$$

In [23]:
# Each symmetry is separated with an underscore
TDDDD = ixp.declarerank4('TDDDD', 'anti03_sym12', DIM=3)

In [24]:
print("Anti-symmetry example:")
print(f"TDDDD[0][1][2][1] =  {TDDDD[0][1][2][1]}")
print(f"TDDDD[1][1][2][0] = {TDDDD[1][1][2][0]}")
print("Symmetry example:")
print(f"TDDDD[0][1][2][1] = {TDDDD[0][1][2][1]}")
print(f"TDDDD[0][2][1][1] = {TDDDD[0][2][1][1]}")

Anti-symmetry example:
TDDDD[0][1][2][1] =  TDDDD0121
TDDDD[1][1][2][0] = -TDDDD0121
Symmetry example:
TDDDD[0][1][2][1] = TDDDD0121
TDDDD[0][2][1][1] = TDDDD0121


If you wish to declare indexed expressions with ranks higher than $4$, you can use the function
```
ixp.declare_indexedexp(rank, symbol=None, symmetry=None, dimension=None)
```

In [25]:
cDDDUUU = ixp.declare_indexedexp(rank=6, symbol="cDDDUUU", symmetry=None, dimension=3)

In [26]:
print(cDDDUUU[0][2][1][2][1][0])

cDDDUUU021210


<a id='indexed_expressions_combined_example'></a>

## Step 3.a: Example using indexed expressions and reference metrics \[Back to [top](#toc)\]
$$\label{indexed_expressions_combined_example}$$

Let us now combine the things we have learned so far in an example. We will compute the Christoffel symbols ourselves and check if they agree with `rfm.GammaUDD`.

The Christoffel symbols are given by

$$
    \hat{\Gamma}^{i}_{\ j k} = \frac{1}{2} \hat{g}^{l i} 
           ( \hat{g}_{l j, k} + \hat{g}_{l k, j} - \hat{g}_{j k, l} ) \,.
$$

The hat diacritic denotes a flat metric.

In [27]:
# First, define rank-3 indexed expression
GammahatUDD = ixp.zerorank3(DIM=3)
for i in range(3):
    for j in range(3):
        for k in range(3):
            for l in range(3):
                GammahatUDD[i][j][k] += sp.simplify( sp.Rational(1,2) * rfm.ghatUU[l][i] * (
                                             rfm.ghatDD[l][j].diff(rfm.xx[k]) +
                                             rfm.ghatDD[l][k].diff(rfm.xx[j]) -
                                             rfm.ghatDD[j][k].diff(rfm.xx[l])
                                             ))

$\hat{g}_{l j, k}$ is computed as `rfm.ghatDD[l][j].diff(rfm.xx[k])`. Refer to [sympy.diff](https://docs.sympy.org/latest/tutorials/intro-tutorial/calculus.html#derivatives) in the official `SymPy` documentation for more details.

In [28]:
# Comparing Christoffel symbols
for i in range(3):
    for j in range(3):
        for k in range(j,3):
            print (sp.simplify(GammahatUDD[i][j][k] - rfm.GammahatUDD[i][j][k]))

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


<a id='fd_derivatives'></a>

# Step 4: Finite-difference derivatives \[Back to [top](#toc)\]
$$\label{fd_derivatives}$$

To denote derivatives, we first append `_d` at the end of the variable name. Subsequently, we append a character `D` for each derivative order.

**Examples**:

* $a^{i}_{\ ,j} \leftrightarrow$ `aU_dD`


* $b_{i,jk} \leftrightarrow$ `bD_dDD`


* $g_{ij, kl} \leftrightarrow$ `gDD_dDD`


* $\Gamma^{i}_{ \ jk,l} \leftrightarrow$ `GammaUDD_dD`


* $R^{i}_{ \ j k l m, pq} \leftrightarrow$ `RUDDDD_dDD`

Notice that, if we define a indexed expression using the above notation, nothing different will happen:

In [29]:
aU_dD = ixp.declarerank2('aU_dD', 'nosym', DIM=3)

In [30]:
aU_dD

[[aU_dD00, aU_dD01, aU_dD02],
 [aU_dD10, aU_dD11, aU_dD12],
 [aU_dD20, aU_dD21, aU_dD22]]

However, this convention will be required when generating C code.

<a id='c_code_generation'></a>

# Step 5: Generating C code \[Back to [top](#toc)\]
$$\label{c_code_generation}$$

The Laplacian of a function $u$ can be express in a covariant way as

$$
\begin{aligned}
\hat{\nabla}^2 u &= \hat{\gamma}^{ij} \hat{\nabla}_j \hat{\nabla}_i u \\
                   &=  \hat{\gamma}^{ij} (\hat{\nabla}_j u_{,i}) \\
                   &= \hat{\gamma}^{ij} ( u_{,ij} - u_{,k} \hat{\Gamma}^{k}_{\ \ ij} ) \\
                   &= \hat{\gamma}^{ij} u_{,ij} - ( \hat{\gamma}^{ij}\hat{\Gamma}^{k}_{\ \ ij}) u_{,k} \\
                   &= \hat{\gamma}^{ij} u_{,ij}  - \hat{\Gamma}^{i} u_{,i} \,.
\end{aligned}
$$

In the last equation, I replaced the summation index to $i$. Moreover,  we have defined the contracted Christoffel symbols as

$$
\hat{\Gamma}^{k} = \hat{\gamma}^{ij}\hat{\Gamma}^{k}_{\ \ ij} \,.
$$


In the next two examples, we will generate C codes to evaluate these expression with finite-difference derivatives.

<a id='c_code_generation_spherical_symmetry'></a>

## Step 5.a: Laplacian with spherical symmetry \[Back to [top](#toc)\]
$$\label{c_code_generation_spherical_symmetry}$$

In [31]:
# Step 0: Read the spatial dimension parameter as DIM.
DIM = par.parval_from_str("grid::DIM")

# Step 1a: Set the finite differencing order to 4.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER",4)

# Step 1b: Choose symmetry axes as '12', all derivatives _dD1, _dD2, _dDD1*, or _dDD2* vanish
par.set_parval_from_str("indexedexp::symmetry_axes",'12')

# Step 2a: Reset the gridfunctions list
gri.glb_gridfcs_list = []

# Step 2b: Register gridfunction
uu = gri.register_gridfunctions("EVOL",["uu"])

# Step 3a: Declare first derivative
uu_dD = ixp.declarerank1("uu_dD")

# Step 3b: Declare second derivative
uu_dDD = ixp.declarerank2("uu_dDD","sym01")

$$
\hat{\Gamma}^{k} = \hat{\gamma}^{ij}\hat{\Gamma}^{k}_{\ \ ij} \,.
$$

In [32]:
# Step 4: Compute contracted Christoffel symbols
contractedGammahatU = ixp.zerorank1()
for k in range(3):
    for i in range(3):
        for j in range(3):
            contractedGammahatU[k] += rfm.ghatUU[i][j] * rfm.GammahatUDD[k][i][j]

$$
\hat{\nabla}^2 u = {\underbrace {\textstyle \hat{\gamma}^{ij} u_{,ij}}_{\text{Part 1}}}  
                 - {\underbrace {\textstyle \hat{\Gamma}^{i} u_{,i}}_{\text{Part 2}}} \,.
$$

In [33]:
# Step 5: Compute Laplacian of u
u_laplacian = sp.sympify(0)
for i in range(DIM):
    # PART 2:
    u_laplacian -= contractedGammahatU[i]*uu_dD[i]
    for j in range(DIM):
        # PART 1:
        u_laplacian += rfm.ghatUU[i][j]*uu_dDD[i][j]

In [34]:
# Step 6: Generate C code for Laplacian
print(fin.FD_outputC("returnstring",
        [lhrh(lhs="u_laplacian",rhs=u_laplacian)]).replace(",i1,i2","").replace("_i1_i2","") )

{
  /*
   * NRPy+ Finite Difference Code Generation, Step 1 of 2: Read from main memory and compute finite difference stencils:
   */
  /*
   *  Original SymPy expressions:
   *  "[const double uu_dD0 = invdx0*(-2*uu_i0m1/3 + uu_i0m2/12 + 2*uu_i0p1/3 - uu_i0p2/12),
   *    const double uu_dDD00 = invdx0**2*(-5*uu/2 + 4*uu_i0m1/3 - uu_i0m2/12 + 4*uu_i0p1/3 - uu_i0p2/12)]"
   */
  const double uu_i0m2 = in_gfs[IDX4S(UUGF, i0-2)];
  const double uu_i0m1 = in_gfs[IDX4S(UUGF, i0-1)];
  const double uu = in_gfs[IDX4S(UUGF, i0)];
  const double uu_i0p1 = in_gfs[IDX4S(UUGF, i0+1)];
  const double uu_i0p2 = in_gfs[IDX4S(UUGF, i0+2)];
  const double FDPart1_Rational_2_3 = 2.0/3.0;
  const double FDPart1_Rational_1_12 = 1.0/12.0;
  const double FDPart1_Rational_5_2 = 5.0/2.0;
  const double FDPart1_Rational_4_3 = 4.0/3.0;
  const double uu_dD0 = invdx0*(FDPart1_Rational_1_12*(uu_i0m2 - uu_i0p2) + FDPart1_Rational_2_3*(-uu_i0m1 + uu_i0p1));
  const double uu_dDD00 = ((invdx0)*(invdx0))*(FDPart1_Ra

<a id='c_code_generation_no_symmetry'></a>

## Step 5.b: Laplacian without symmetry \[Back to [top](#toc)\]
$$\label{c_code_generation_no_symmetry}$$

In [35]:
# Choose a different coordinate system
par.set_parval_from_str("reference_metric::CoordSystem", "SinhSymTP")
# And set the dimension to 3
par.set_parval_from_str("grid::DIM",3)
# Recompute geometric quantities in the new coordinate system
rfm.reference_metric()

In [36]:
# Step 1a: Set the finite differencing order to 10.
par.set_parval_from_str("finite_difference::FD_CENTDERIVS_ORDER", 10)

# Step 1b: Remove symmmetry axes
par.set_parval_from_str("indexedexp::symmetry_axes",'nosym')

# Step 2a: Reset the gridfunctions list
gri.glb_gridfcs_list = []

# Step 2b: Register gridfunction
uu = gri.register_gridfunctions("EVOL",["uu"])

# Step 3a: Declare first derivative
uu_dD = ixp.declarerank1("uu_dD")

# Step 3b: Declare second derivative
uu_dDD = ixp.declarerank2("uu_dDD","sym01")

# Step 4: Compute contracted Christoffel symbols
contractedGammahatU = ixp.zerorank1()
for k in range(3):
    for i in range(3):
        for j in range(3):
            contractedGammahatU[k] += rfm.ghatUU[i][j] * rfm.GammahatUDD[k][i][j]

# Step 5: Compute Laplacian of u
u_laplacian = sp.sympify(0)
for i in range(DIM):
    # PART 2:
    u_laplacian -= contractedGammahatU[i]*uu_dD[i]
    for j in range(DIM):
        # PART 1:
        u_laplacian += rfm.ghatUU[i][j]*uu_dDD[i][j]

In [37]:
# Step 6: Generate C code for Laplacian
print(fin.FD_outputC("returnstring",
        [lhrh(lhs="u_laplacian",rhs=u_laplacian)],
        params="CSE_enable=True,enable_SIMD=False,outCverbose=False") )

{
  /*
   * NRPy+ Finite Difference Code Generation, Step 1 of 2: Read from main memory and compute finite difference stencils:
   */
  const double uu_i0_i1_i2m5 = in_gfs[IDX4S(UUGF, i0,i1,i2-5)];
  const double uu_i0_i1_i2m4 = in_gfs[IDX4S(UUGF, i0,i1,i2-4)];
  const double uu_i0_i1_i2m3 = in_gfs[IDX4S(UUGF, i0,i1,i2-3)];
  const double uu_i0_i1_i2m2 = in_gfs[IDX4S(UUGF, i0,i1,i2-2)];
  const double uu_i0_i1_i2m1 = in_gfs[IDX4S(UUGF, i0,i1,i2-1)];
  const double uu_i0_i1m5_i2 = in_gfs[IDX4S(UUGF, i0,i1-5,i2)];
  const double uu_i0_i1m4_i2 = in_gfs[IDX4S(UUGF, i0,i1-4,i2)];
  const double uu_i0_i1m3_i2 = in_gfs[IDX4S(UUGF, i0,i1-3,i2)];
  const double uu_i0_i1m2_i2 = in_gfs[IDX4S(UUGF, i0,i1-2,i2)];
  const double uu_i0_i1m1_i2 = in_gfs[IDX4S(UUGF, i0,i1-1,i2)];
  const double uu_i0m5_i1_i2 = in_gfs[IDX4S(UUGF, i0-5,i1,i2)];
  const double uu_i0m4_i1_i2 = in_gfs[IDX4S(UUGF, i0-4,i1,i2)];
  const double uu_i0m3_i1_i2 = in_gfs[IDX4S(UUGF, i0-3,i1,i2)];
  const double uu_i0m2_i1_i2 = in_

In [38]:
# Step 7: Generate C code for Laplacian with SIMD compiler extrinsics
print(fin.FD_outputC("returnstring",
        [lhrh(lhs="u_laplacian",rhs=u_laplacian)],
        params="CSE_enable=True,enable_SIMD=True,outCverbose=False") )

{
  /*
   * NRPy+ Finite Difference Code Generation, Step 1 of 2: Read from main memory and compute finite difference stencils:
   */
  const REAL_SIMD_ARRAY uu_i0_i1_i2m5 = ReadSIMD(&in_gfs[IDX4S(UUGF, i0,i1,i2-5)]);
  const REAL_SIMD_ARRAY uu_i0_i1_i2m4 = ReadSIMD(&in_gfs[IDX4S(UUGF, i0,i1,i2-4)]);
  const REAL_SIMD_ARRAY uu_i0_i1_i2m3 = ReadSIMD(&in_gfs[IDX4S(UUGF, i0,i1,i2-3)]);
  const REAL_SIMD_ARRAY uu_i0_i1_i2m2 = ReadSIMD(&in_gfs[IDX4S(UUGF, i0,i1,i2-2)]);
  const REAL_SIMD_ARRAY uu_i0_i1_i2m1 = ReadSIMD(&in_gfs[IDX4S(UUGF, i0,i1,i2-1)]);
  const REAL_SIMD_ARRAY uu_i0_i1m5_i2 = ReadSIMD(&in_gfs[IDX4S(UUGF, i0,i1-5,i2)]);
  const REAL_SIMD_ARRAY uu_i0_i1m4_i2 = ReadSIMD(&in_gfs[IDX4S(UUGF, i0,i1-4,i2)]);
  const REAL_SIMD_ARRAY uu_i0_i1m3_i2 = ReadSIMD(&in_gfs[IDX4S(UUGF, i0,i1-3,i2)]);
  const REAL_SIMD_ARRAY uu_i0_i1m2_i2 = ReadSIMD(&in_gfs[IDX4S(UUGF, i0,i1-2,i2)]);
  const REAL_SIMD_ARRAY uu_i0_i1m1_i2 = ReadSIMD(&in_gfs[IDX4S(UUGF, i0,i1-1,i2)]);
  const REAL_SIMD_ARRAY uu

<a id='latex_pdf_output'></a>

# Step 6: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-A-survey-of-some-NRPy-features.pdf](Tutorial-A-survey-of-some-NRPy-features.pdf). (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [39]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-A-survey-of-some-NRPy-features")

Created Tutorial-A-survey-of-some-NRPy-features.tex, and compiled LaTeX
    file to PDF file Tutorial-A-survey-of-some-NRPy-features.pdf
